In [1]:
from collections import Counter, defaultdict
import heapq

In [3]:
# Task Scheduler 

# You are given an array of CPU tasks, each represented by letters A to Z, and a cooling time, n.
# Each cycle or interval allows the completion of one task. Tasks can be completed in any order, but there's a constraint:
#                                           identical tasks must be separated by at least n intervals due to cooling time.
# Return the minimum number of intervals required to complete all tasks.


'''
# Interface
Args:
    tasks: ["A", "C", "B"]
    n: cooling period
def min_intervals_of_completion(tasks, n) -> return num intervals


# Example

(1) AAB, n = 1
BA-A 
ABA ---> return 3

(2) AABBBC,n = 3
rem = B1
BAC-BA--B 

# Algorithm
(1)
Select each char greedy every time.
With heap. # (-cnt, char), next_possible_t
   - I find the task most frequent in each t.
   - 

AAABBC n = 3

(-3,A,1), (-2,B,1), (-1,C,1)
t = 1, get (-3, A)
(-2,B), (-1,C) <---- (-2, A, 5)
t = 2, get (-2, A)

(2)
I will keep track
- heap # (- cnt, task_id)
- tasks_in_cooling_period = {available_time -> task_id[] }


for t = 0~,
    - add tasks of tasks_in_cooling_period[t]
    - pop out one task.
    - add the task to tasks_in_cooling_period

aaaa, n = 10000

time = N * max(tasks) + N logN
space = N
'''

# Impl
def min_intervals_of_completion(tasks, n): # "ABAB", 10
    task_cnt_and_type = [(-cnt, task_type) for task_type, cnt in Counter(tasks).items()] # (-2,A), (-2, B) #[](-1, A)
    heapq.heapify(task_cnt_and_type)
    tasks_in_cooling_period = defaultdict(list) # {task_type -> (-cnt, task_type)} # {} -> {12: (-1, A), 13:(-1, B)}

    t = 0 # 2->3-> 13
    while 0 < len(task_cnt_and_type) or 0 < len(tasks_in_cooling_period):
        t += 1
        for task in tasks_in_cooling_period[t]:
            heapq.heappush(task_cnt_and_type, task)

        ### I forgot to delete this !!!!!!!!!!!
        ### Without this, it loops inifiitely due to `or 0 < len(tasks_in_cooling_period)`
        if t in tasks_in_cooling_period:
            del tasks_in_cooling_period[t]

        if len(task_cnt_and_type) == 0:
            continue

        out = heapq.heappop(task_cnt_and_type)
        cnt, task_type = - out[0], out[1] # 1,  B

        # task is done here.
        rem_cnt = cnt - 1 # 0
        if 1 <= rem_cnt:
            tasks_in_cooling_period[t + n + 1].append((-rem_cnt, task_type))

    return t


# Test

assert min_intervals_of_completion("ABAB", 10) == 13
assert min_intervals_of_completion("AA", 10) == 12
assert min_intervals_of_completion("", 10) == 0 # pass

In [ ]:
# Maximum Number of Events That Can Be Attended

# You are given an array of events where events[i] = [startDayi, endDayi]. Every event i starts at startDayi and ends at endDayi.
# You can attend an event i at any day d where startTimei <= d <= endTimei. You can only attend one event at any time d.
# Return the maximum number of events you can attend.

'''

# Example

[1,5],[1,5],[1,5],[2,3],[2,3]

1------5
1------5
1------5
  2-3
  2-3


curr_day = 1



sort events

heap = [] # (end_date, start_date)

for each day,
   for each event which starts on that day, push it into heap
   pop out event
       if it is invalid (out.end_date < curr.start_date), ignore it.
       otherwise, cnt += 1
'''

from heapq import heappush, heappop, heapify

# Impl
def attendable_cnt(events):
    events.sort()
    cnt = 0
    
    started_events = [] # (end, start)

    i = 0
    curr_day = 1
    while i < len(events) or 0 < len(started_events):
        # Push all the events started today into the heap `started_events`.
        while i < len(events) and events[i][0] <= curr_day:
            event = events[i]
            heappush(started_events, (event[1], event[0]))
            i += 1

        # Pop out all the outdated events from heap.
        while 0 < len(started_events) and started_events[0][0] < curr_day:
            heappop(started_events)

        # Pop out an event attended today.
        if 0 < len(started_events):
            heappop(started_events)
            cnt += 1

        curr_day += 1
    return cnt


'''
time  = NlogN + max(endDate)
space = N

'''

# Test
print(attendable_cnt([[2,2],[1,2], [1,2]]))
assert attendable_cnt([[2,2],[1,2], [1,2]]) == 2
assert attendable_cnt([[2,2],[1,2]]) == 2
assert attendable_cnt([]) == 0
assert attendable_cnt([[1,5],[1,5],[1,5],[2,3],[2,3]]) == 5

In [ ]:
# Reorganize string

'''
aab -> aba
{a: 2, b: 1}
-> use "a" (the most cnt)
{a: 1, b: 1}
-> use "b" (not a)
{a: 1, b: 0}
-> use "a" (the most cnt)

Use min heap.
h = [] # (- cnt, char)

count each char.
build heap (-cnt, char)

keep track of chars = []
while 0 < len(heap)
    pop out from heap(1)
    if out_char != prev_char,
        decrement cnt and push it back.
        append to chars
    otherwise,
        pop one more(2). if there is not any, return ""
        append (2) to chars
        push back (1) and cnt-decrmented (2)

joined chars

Assuming N=len(s) and K=nunique(s),
time  = N log K
space = K

'''

class Solution:
    def reorganizeString(self, s: str) -> str:
        # (-rem_cnt, char)
        rem_chars = [(-cnt, char) for char, cnt in Counter(s).items()]
        heapq.heapify(rem_chars)

        selected_chars = []

        while 0 < len(rem_chars):
            out = heapq.heappop(rem_chars)
            rem_cnt, char = - out[0], out[1]

            if len(selected_chars) == 0 or selected_chars[-1] != char:
                selected_chars.append(char)
                rem_cnt -= 1
                if 0 < rem_cnt:
                    heapq.heappush(rem_chars, (-rem_cnt, char))
            else:
                if len(rem_chars) == 0:
                    return ""
                out2 = heapq.heappop(rem_chars)
                rem_cnt2, char2 = - out2[0], out2[1]
                assert selected_chars[-1] != char2

                selected_chars.append(char2)

                heapq.heappush(rem_chars, (-rem_cnt, char))

                rem_cnt2 -= 1
                if 0 < rem_cnt2:
                    heapq.heappush(rem_chars, (-rem_cnt2, char2))
        
        return "".join(selected_chars)



In [20]:


# A string s is called happy if it satisfies the following conditions:

# s only contains the letters 'a', 'b', and 'c'.
# s does not contain any of "aaa", "bbb", or "ccc" as a substring.
# s contains at most a occurrences of the letter 'a'.
# s contains at most b occurrences of the letter 'b'.
# s contains at most c occurrences of the letter 'c'.
# Given three integers a, b, and c, return the longest possible happy string. If there are multiple longest happy strings, return any of them.
# If there is no such string, return the empty string "".

# A substring is a contiguous sequence of characters within a string.

'''
# interface
def longeset_happy_string(a, b, c) -> return string

- a, b, c is integer. 0 <= a,b,c.

# example
a,b,c = 2,3,4

aabbbcccc   x
aabccbbcc   o

a,b,c = 2,2,100
cc a cc b cc a cc b cc
{a2b2c99}-> {a2b2c98} -> {a1b2c98}
cc a cc

# algorithm

Track
- second_last_char, last_chars
- rem_char_to_cnt (list of (- frequency, char))
- chars

while len(rem_char_to_cnt):
    pop element. 
    if out = second_last_char = last_chars, pop again.
    append used one to char.
    decremnt frequency and push it back.
    for the other, push it back
return contanated chars.

time  = A + B + C
space = A + B + C
'''
# Impl
def longeset_happy_string(a, b, c): # a8, b1, c0
    # x(-8, a), (-1, b), (-7, a)
    rem_chars = [(-cnt, char) for char, cnt in [("a", a), ("b", b), ("c", c)] if 0 < cnt] # x(-8, a), (-1, b), x(-7, a),  (-6, a), ()
    heapq.heapify(rem_chars)
    chars = [] # a, a, b
    second_last_char, last_char = None, None # a, a

    while 0 < len(rem_chars):
        out1 = heapq.heappop(rem_chars)
        cnt1, char1 = - out1[0], out1[1] # 6, a

        #  
        if second_last_char == last_char == char1:
            if len(rem_chars) == 0:
                break
            out2 = heapq.heappop(rem_chars)
            cnt2, char2 = - out2[0], out2[1] # 1, b

            chars.append(char2)
            heapq.heappush(rem_chars, (-cnt1, char1))
            second_last_char, last_char = last_char, char2
            if 2 <= cnt2:
                heapq.heappush(rem_chars, (-cnt2+1, char2))
        else:
            chars.append(char1)
            if 2 <= cnt1:
                heapq.heappush(rem_chars, (-cnt1+1, char1))
            second_last_char, last_char = last_char, char1

    return "".join(chars)
    
# Test
print(longeset_happy_string(8,1,1))
assert longeset_happy_string(8,1,1) in ["aabaacaa", "aacaabaa"]
assert longeset_happy_string(8,1,0) == "aabaa"
assert longeset_happy_string(8,0,0) == "aa"
assert longeset_happy_string(0,0,0) == ""

aabaacaa
